In [ ]:
tf.enable_eager_execution()

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

import tensorflow as tf

!pip install -q tensorflow-hub
!pip install -q tensorflow-datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print('Version:',tf.__version__)
print('Eager mode:',tf.executing_eagerly())
print('Hub version:',hub.__version__)
print('GPU is','available' if tf.config.experimental.list_physical_devices('GPU') else 'NOT AVAILABLE')


Version: 1.14.0
Eager mode: True
Hub version: 0.7.0
GPU is NOT AVAILABLE


In [8]:
# Download the IMDB dataset
# Split the training set into 60% and 40%, so we'll end up with
# 15,000 examples for training, 10,000 examples for validation
# and 25,000 examples for testing
train_validation_split = tfds.Split.TRAIN.subsplit([6,4])
print(train_validation_split)
(train_data, validation_data),test_data = tfds.load(
    name='imdb_reviews',
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True
)

(NamedSplit('train')(tfds.percent[0:60]), NamedSplit('train')(tfds.percent[60:100]))


KeyboardInterrupt: 

In [ ]:
# Explore the data
# lets print the first 10 examples

train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

In [ ]:
# print the first 10 labels
train_labels_batch

In [ ]:
# Build the Model
embedding = 'https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1'
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                          dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

In [ ]:
# the full model
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
# compile the model
model.compile(optimizer='adam',loss='binary_crossentopy',merics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    train_data.shuffle(10000).batch(512),
    epochs=20,
    validation_data=validation_data.batch(512),
    verbose=1
)

In [ ]:
# Evaluate the model
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print('%s: %.3f' %(name, value))